**Общая информация**

**Срок сдачи:** 21 октября 2020, 08:30 

**Штраф за опоздание:** по 1 баллу за 24 часа задержки. Через 5 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0220, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
2. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [20]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_20newsgroups

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KDTree

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

In [37]:
from scipy.spatial import distance_matrix
from scipy.stats import mode

class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors, algorithm='brute'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm
    
    def fit(self, X, y):
        self.X_train = np.asarray(X)
        self.y_train = np.asarray(y)
        return self
    
    def predict_proba(self, X):
        X = np.asarray(X)
        
        if self.algorithm == 'brute':
            dst_mtrx = distance_matrix(X, self.X_train)
            # indices of min n_neighbors dists for every x in X_test
            indices = np.argpartition(dst_mtrx, 
                                      kth=self.n_neighbors, 
                                      axis=1)[:, :self.n_neighbors]
        elif self.algorithm == 'kd_tree':
            tree = KDTree(self.X_train)
            min_dsts, indices = tree.query(X, k=self.n_neighbors)
            
        # classes for min n_neighbors dists for every x in X_test
        y_proba = np.take_along_axis(self.y_train[np.newaxis, :], 
                                     indices, 
                                     axis=1)        
        return y_proba
    
    def predict(self, X):
        y_proba = self.predict_proba(X) 
        modes, _ = mode(y_proba, axis=1)
        y_pred = modes.ravel()
        return y_pred

In [35]:
rng = np.random.RandomState(0)
X = rng.random_sample((10, 3))
tree = KDTree(X)
dist, ind = tree.query(X, k=3)
dist, ind
# X[0][np.newaxis, :], X[:1]

(array([[0.        , 0.19662693, 0.29473397],
        [0.        , 0.29019522, 0.29473397],
        [0.        , 0.34159159, 0.41689499],
        [0.        , 0.19662693, 0.41860234],
        [0.        , 0.51150232, 0.53771137],
        [0.        , 0.63809564, 0.83929158],
        [0.        , 0.34159159, 0.4545101 ],
        [0.        , 0.29019522, 0.398299  ],
        [0.        , 0.49193536, 0.53771137],
        [0.        , 0.3983422 , 0.47216408]]),
 array([[0, 3, 1],
        [1, 7, 0],
        [2, 6, 0],
        [3, 0, 1],
        [4, 3, 8],
        [5, 1, 7],
        [6, 2, 7],
        [7, 1, 0],
        [8, 3, 4],
        [9, 7, 1]]))

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [38]:
iris = datasets.load_iris()

In [39]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [40]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [41]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

MyKNeighborsClassifier(n_neighbors=2)

In [42]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred) )<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [43]:
%time clf.fit(X_train, y_train)

CPU times: user 903 µs, sys: 179 µs, total: 1.08 ms
Wall time: 828 µs


KNeighborsClassifier(algorithm='brute', n_neighbors=2)

In [44]:
%time my_clf.fit(X_train, y_train)

CPU times: user 19 µs, sys: 4 µs, total: 23 µs
Wall time: 29.8 µs


MyKNeighborsClassifier(n_neighbors=2)

In [45]:
%time clf.predict(X_test)

CPU times: user 4.84 ms, sys: 964 µs, total: 5.8 ms
Wall time: 5.71 ms


array([1, 0, 2, 0, 1, 2, 0, 0, 1, 2, 1, 2, 1, 0, 2])

In [46]:
%time my_clf.predict(X_test)

CPU times: user 2.59 ms, sys: 0 ns, total: 2.59 ms
Wall time: 1.48 ms


array([1, 0, 2, 0, 1, 2, 0, 0, 1, 2, 1, 2, 1, 0, 2])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [47]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [48]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [49]:
%time clf.fit(X_train, y_train)

CPU times: user 1.27 ms, sys: 0 ns, total: 1.27 ms
Wall time: 818 µs


KNeighborsClassifier(algorithm='kd_tree', n_neighbors=2)

In [50]:
%time my_clf.fit(X_train, y_train)

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 10.3 µs


MyKNeighborsClassifier(algorithm='kd_tree', n_neighbors=2)

In [51]:
%time clf.predict(X_test)

CPU times: user 2.88 ms, sys: 0 ns, total: 2.88 ms
Wall time: 2.21 ms


array([2, 0, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 0, 0, 1])

In [52]:
%time my_clf.predict(X_test)

CPU times: user 2.37 ms, sys: 0 ns, total: 2.37 ms
Wall time: 1.81 ms


array([2, 0, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 0, 0, 1])

In [53]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [45]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [46]:
data = newsgroups['data']
target = newsgroups['target']

Преобразуйте текстовые данные из data с помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Словарь можно ограничить по частотности.

In [58]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data)

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальную точность в среднем на валидации на 3 фолдах).
Постройте график зависимости средней точности от количества соседей. Можно рассмотреть число соседей от 1 до 10.

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf векторизацию( sklearn.feature_extraction.text.TfidfVectorizer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим точность вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.